In [3]:
!pip install refractml==1.0.3 --target /packages/Python-3.8/de678bea-42c4-42fe-88b0-04bfdecd1b35/3.8 --upgrade
# !pip install scikit-learn==0.24.2

  Using cached https://files.pythonhosted.org/packages/f9/0d/023d845cf453feb632b08435c91f7e5d050c0df73c5be66bdbbca2f6ba87/refractml-1.0.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/ef/7681134338fc097acef8d9b2f8abe0458e4d87559c689a8c306d0957ece5/requests_toolbelt-0.9.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d7/42/7ad4b6d67a16229496d4f6e74201bdbebcf4bc1e87d5a70c9297d4961bd2/PyYAML-6.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/09/d7/8424b1dcaa5b1a2f824fc440aa1c4ef45e0bf6593d11b37962311614f365/mosaic_utils-1.0.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7b/f5/890a0baca17a61c1f92f72b81d3c31523c99bec609e60c292ea55b387ae8/urllib3-1.26.15-py2.py3-none-any.whl
Processing /home/mosaic-ai/.cache/pip/wheels/ab/d0/0e/613976a1b51b5654859e2a82ade64329859bce431e280f2a39/shutils-0.1.0-cp38-

In [ ]:
# !pip uninstall urllib3 -y

In [ ]:
# !pip install urllib3==1.26.15

In [1]:
import pandas as pd
import numpy as np
# import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
from refractml import *
from refractml.constants import MLModelFlavours

ImportError: libopenblasp-r0-41284840.3.18.so: cannot open shared object file: No such file or directory

In [2]:
!find / -name "libopenblasp-r0-41284840.3.18.so"

/tmp/pip_packages/scipy.libs/libopenblasp-r0-41284840.3.18.so
find: ‘/home/miniconda’: Permission denied
find: ‘/proc/tty/driver’: Permission denied
find: ‘/lost+found’: Permission denied
find: ‘/var/lib/rhsm’: Permission denied
find: ‘/var/lib/private’: Permission denied
find: ‘/var/db/sudo’: Permission denied
find: ‘/var/cache/ldconfig’: Permission denied
find: ‘/root’: Permission denied
find: ‘/etc/sudoers.d’: Permission denied
/packages/Python-3.8/7ba5239c-5fc2-48ab-8a2b-5408f2ab126b/3.8/scipy.libs/libopenblasp-r0-41284840.3.18.so
/packages/Python-3.8/696b47a1-e8f4-4207-aba3-c50d2a847703/3.8/scipy.libs/libopenblasp-r0-41284840.3.18.so
/packages/Python-3.8/c687b99d-7dc0-4bbf-b1fc-a307bc39851e/3.8/scipy.libs/libopenblasp-r0-41284840.3.18.so
/packages/Python-3.8/a0a61d35-777c-48a1-bf65-dbf5a988374d/3.8/scipy.libs/libopenblasp-r0-41284840.3.18.so
/packages/Python-3.8/480cbb8a-576b-4c95-be08-18b7b6fd97f5/3.8/scipy.libs/libopenblasp-r0-41284840.3.18.so
/packages/Python-3.8/3a4ff4e6-7857-

In [ ]:
cars = pd.read_csv("car_new.csv")
cars

In [ ]:
Y = cars['class_num']
X = cars.drop (['class_num'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

In [ ]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [ ]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.Series(model.predict(data))
    return prediction.tolist()

In [ ]:
req = requests.Request()
req.json = {"payload":X_train.head(1).to_json()}

sch = generate_schema(score,(model, req), X_train)
print(sch)

In [ ]:
!pip show scikit-learn

In [ ]:
register_model(model, 
               score, 
               name='Refractml_Sklearn_Python_38_test_packages', 
               description='car evaluation classification model using sklearn', 
               flavour=MLModelFlavours.sklearn,
               schema=sch,
               y_true=y_test, 
               y_pred=y_pred,
               y_train=y_train, 
               y_test=y_test,
               x_train=X_train, 
               x_test=X_test, 
               prob=y_prob,
               labels=[0,1], 
               init_script="",
               model_type="classification",
               input_type="json", 
               explain_ai=False, 
               feature_names=list(X_train), 
               feature_ids=list(X_train),
               target_names=[0,1,2,3], 
               kyd=True, 
               kyd_score = False
              )